In [1]:
import json
import os
import uuid
from datetime import datetime, timezone
from pymongo import MongoClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
collection_name = "inventories"

# Document-level timestamp
doc_created = datetime.fromisoformat("2025-10-12T08:00:00+00:00")

# Base item data
base_items = [
    {"barcode": "4809992000101", "type": "fuel", "name": "Premium Gasoline", "price": 69.50, "unit": "liter", "previous_quantity": 520, "addstock": 300, "sold": 250, "current_quantity": 570},
    {"barcode": "4809992000102", "type": "fuel", "name": "Regular Gasoline", "price": 64.75, "unit": "liter", "previous_quantity": 680, "addstock": 400, "sold": 320, "current_quantity": 760},
    {"barcode": "4809992000103", "type": "fuel", "name": "Premium Diesel", "price": 61.20, "unit": "liter", "previous_quantity": 450, "addstock": 350, "sold": 280, "current_quantity": 520},
    {"barcode": "4809992000201", "type": "lubricant", "name": "Engine Oil SAE 40", "price": 250.00, "unit": "liter", "previous_quantity": 60, "addstock": 40, "sold": 25, "current_quantity": 75},
    {"barcode": "4809992000202", "type": "lubricant", "name": "Automatic Transmission Fluid (ATF)", "price": 310.00, "unit": "liter", "previous_quantity": 50, "addstock": 30, "sold": 20, "current_quantity": 60},
    {"barcode": "4809992000203", "type": "lubricant", "name": "Gear Oil EP 90", "price": 285.00, "unit": "liter", "previous_quantity": 45, "addstock": 25, "sold": 15, "current_quantity": 55},
    {"barcode": "4809992000204", "type": "lubricant", "name": "Hydraulic Oil ISO 68", "price": 295.00, "unit": "liter", "previous_quantity": 35, "addstock": 25, "sold": 10, "current_quantity": 50},
    {"barcode": "4809992000205", "type": "lubricant", "name": "2T Motorcycle Oil", "price": 180.00, "unit": "liter", "previous_quantity": 80, "addstock": 60, "sold": 45, "current_quantity": 95},
]

# Add item_number and other metadata
items = [
    {
        "item_number": str(i + 1),
        "id": str(uuid.uuid4()),
        "created": doc_created,
        "updated": doc_created,
        **item
    }
    for i, item in enumerate(base_items)
]

# Full inventory document
doc = {
    "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
    "previous_form_qr_code": "20251011",
    "current_form_qr_code": "20251012",
    "date": "2025-10-12",
    "location": "JEF Gas Station – Sikatuna Branch",
    "tin": "123-456-789",
    "branch": "JEF Biosciences Fuel Division",
    "created": doc_created,
    "items": items,
    "cashier": "Liam Santos",
    "cashier_employee_number": "50321",
    "recorder": "Ella Reyes",
    "recorder_employee_number": "50325",
    "is_empty": False
}

# MongoDB connection
client = MongoClient(uri)
db = client[database]
coll = db[collection_name]

# Ensure 'id' is unique
coll.create_index("id", unique=True)

# Insert or update document
res = coll.replace_one({"id": doc["id"]}, doc, upsert=True)

# Output summary
print({
    "acknowledged": res.acknowledged,
    "matched_count": res.matched_count,
    "modified_count": res.modified_count,
    "upserted_id": str(res.upserted_id) if res.upserted_id else None,
    "collection": collection_name
})


{'acknowledged': True, 'matched_count': 0, 'modified_count': 0, 'upserted_id': '68ee7385a268c4061366bb56', 'collection': 'inventories'}
